In [1]:
sealed trait RingF[+A]
case object Zero extends RingF[Nothing]
case object One extends RingF[Nothing]
case class Elem(x: Int) extends RingF[Nothing]
case class Add[A](x: A, y: A) extends RingF[A]
case class Mult[A](x:A, y: A) extends RingF[A]

trait Functor[F[_]] {
    def map[A,B](f: A => B): F[A] => F[B]
}

implicit val ringFunctor = new Functor[RingF] {
    override def map[A, B](f: A => B): RingF[A] => RingF[B] = {
        case Zero => Zero
        case One => One
        case Elem(x) => Elem(x)
        case Add(x, y) => Add(f(x), f(y))
        case Mult(x, y) => Mult(f(x), f(y))
    }
}

case class Fix[F[_]](value: F[Fix[F]] )
object Fix {
  def fix[F[_]](ff: F[Fix[F]]): Fix[F] = new Fix[F](ff)
  def unfix[F[_]]: Fix[F] => F[Fix[F]] = f => f.value
}

type Ring = Fix[RingF]
val zero = Fix[RingF](Zero)
val one =  Fix[RingF](One)
def elem: Int  => Ring = x => Fix[RingF](Elem(x))
def add: (Ring, Ring) => Ring = (x, y) =>  Fix[RingF](Add(x, y))
def mult: (Ring, Ring) => Ring = (x, y) =>  Fix[RingF](Mult(x, y))

defined trait RingF
defined object Zero
defined object One
defined class Elem
defined class Add
defined class Mult
defined trait Functor
ringFunctor: AnyRef with Functor[RingF] = ammonite.$sess.cmd0$Helper$$anon$1@59c96db2
defined class Fix
defined object Fix
defined type Ring
zero: Fix[RingF] = Fix(Zero)
one: Fix[RingF] = Fix(One)
defined function elem
defined function add
defined function mult

In [6]:
def cata[F[_], A](alg: F[A] => A)(implicit F: Functor[F]): Fix[F] => A = {
    x => alg(F.map(cata(alg))(Fix.unfix(x)))
}
def ana[F[_], A](coalg: A => F[A])(implicit F: Functor[F]): A => Fix[F] = {
    x => Fix((F.map(ana(coalg))(coalg(x))))
}

implicit class RingRich(x: Ring) {
    def ringcata[A](alg: RingF[A] => A) = {
        cata(alg)(ringFunctor)(this.x)
    }
}

implicit class Rich[A](x: A) {
    val y = x
    def ringana[A](coalg: A => RingF[A]) = {
        ana(coalg)(ringFunctor)(y.asInstanceOf[A])
    }
}
def evalToInt: RingF[Int] => Int = {
    x => x match {
        case Zero => 0
        case One => 1
        case Elem(x) => x
        case Add(x, y) => x + y
        case Mult(x, y) => x * y
    }
}
def findDivisorsOf(r: Int): RingF[Int] = {
    def loop(n: Int): RingF[Int] = {
        n match {
            case 0 => Zero
            case 1 => One
            case x => 
                if (x >= r) Elem(x)
                else if (r%x == 0) Mult(r/x, x) 
                else loop(n+1)
        }
    }
    loop(2)
}
def prettyPrint: RingF[String] => String = {
    case Zero => "0"
    case One => "1"
    case Elem(x: Int) => x.toString
    case Add(x, y) => x + " + " + y
    case Mult(x, y) => x + " * " + y
}

defined function cata
defined function ana
defined class RingRich
defined class Rich
defined function evalToInt
defined function findDivisorsOf
defined function prettyPrint

In [15]:
add(elem(1), add(elem(2), elem(3))).ringcata(evalToInt)

res14: Int = 6

In [7]:
12.ringana(findDivisorsOf).ringcata(prettyPrint)

res6: String = "3 * 2 * 2"